In [23]:
import pandas as pd 
import numpy as np 

from sklearn import preprocessing
from sklearn import metrics

from sklearn.utils import resample

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.combine import SMOTEENN

from imblearn.ensemble import BalancedBaggingClassifier

import matplotlib.pyplot as plt 
import seaborn as sns

In [24]:
df= pd.read_csv("Training_Dataset.csv")

In [25]:
df.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar48,mvar49,mvar50,mvar51
0,1,Charge,0,106920,4,18,970,0.7969,0.1831,337236,...,0,0,0,0,0,0,1,0,0,0
1,2,Charge,2,0,4,19,1030,0.0678,0.0000,2918974,...,0,0,0,0,1,0,0,0,0,0
2,3,Charge,2,399168,4,35,327,0.7551,0.2986,498656,...,0,0,0,0,0,0,1,0,0,0
3,4,Charge,0,267300,4,274,1258,0.7969,0.5297,2480074,...,0,1,0,0,0,0,1,0,0,1
4,5,Charge,0,213840,6,1,0,2.3238,0.0000,2918974,...,0,0,0,0,1,0,0,1,0,0


In [26]:
spend = df.mvar36 + df.mvar37 + df.mvar38 + df.mvar39

In [27]:
income = df["mvar9"][df.mvar9>0]

In [28]:
income= income.sort_values(ascending=True).reset_index()

In [29]:
median_income= income.median()[1]

In [30]:
mean_income = income.mean()[1]

In [31]:
df.mvar9.replace(to_replace=0, value=mean_income, inplace=True)

In [32]:
df2=df
df2.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar48,mvar49,mvar50,mvar51
0,1,Charge,0,106920,4,18,970,0.7969,0.1831,337236.0,...,0,0,0,0,0,0,1,0,0,0
1,2,Charge,2,0,4,19,1030,0.0678,0.0000,2918974.0,...,0,0,0,0,1,0,0,0,0,0
2,3,Charge,2,399168,4,35,327,0.7551,0.2986,498656.0,...,0,0,0,0,0,0,1,0,0,0
3,4,Charge,0,267300,4,274,1258,0.7969,0.5297,2480074.0,...,0,1,0,0,0,0,1,0,0,1
4,5,Charge,0,213840,6,1,0,2.3238,0.0000,2918974.0,...,0,0,0,0,1,0,0,1,0,0


In [33]:
df2.drop(["cm_key", "mvar1", "mvar3"], axis=1, inplace=True)

In [34]:
df2a= df2.ix[:, 0:13]
df2a.head()

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15
0,0,4,18,970,0.7969,0.1831,337236.0,1,6.0899,Office supplies,12,1,0
1,2,4,19,1030,0.0678,0.0000,2918974.0,1,4.2016,Unknown,11,1,0
2,2,4,35,327,0.7551,0.2986,498656.0,0,3.3760,Misc services,12,1,1
3,0,4,274,1258,0.7969,0.5297,2480074.0,1,4.1082,Social services,12,1,0
4,0,6,1,0,2.3238,0.0000,2918974.0,0,0.0000,Unknown,6,2,0


In [35]:
df2b= df2.ix[:, 37:43]
df2b.head()

,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,1,0,0,1,0,0
4,0,0,0,0,0,0


In [36]:
df3= pd.concat([df2a, df2b], axis=1)

In [37]:
df3["spend"]=spend

In [38]:
# taking log
df3.mvar9 = np.log(df3.mvar9)

# normalizing
# std_scale = preprocessing.StandardScaler().fit(df3.mvar9)
# df3.mvar9 = std_scale.transform(df3.mvar9)

In [40]:
df3.spend = preprocessing.StandardScaler().fit_transform(df3.spend.reshape(-1, 1))

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [42]:
df3.mvar6 = preprocessing.StandardScaler().fit_transform(df3.mvar6.reshape(-1, 1))
# df3.mvar6 = std_scale.transform(df3.mvar6)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [43]:
df3.describe()

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar13,mvar14,mvar15,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,spend
count,40000.000000,40000.000000,40000.000000,4.000000e+04,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,4.000000e+04
mean,1.117150,3.224150,89.264475,5.253575e-17,0.726782,0.167726,13.500402,0.329925,3.384588,11.993125,1.046025,0.329175,0.293025,0.072300,0.070925,0.083075,0.025475,0.004925,-3.393119e-18
std,1.656335,2.324337,92.247710,1.000013e+00,1.262927,0.206095,1.633287,0.470191,1.890469,3.274286,0.422328,0.772550,0.608663,0.315872,0.267014,0.320275,0.218237,0.070006,1.000013e+00
min,0.000000,0.000000,1.000000,-9.843242e-01,0.000100,0.000000,4.094345,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.669039e-01
25%,0.000000,2.000000,16.000000,-6.293233e-01,0.106000,0.000000,11.959003,0.000000,1.753100,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.462064e-01
50%,0.000000,3.000000,53.000000,-3.481451e-01,0.382000,0.115500,13.977593,0.000000,3.220350,12.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.125829e-01
75%,2.000000,4.000000,134.000000,3.151747e-01,0.796900,0.268200,14.886743,1.000000,5.011550,14.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.442737e-02
max,11.000000,12.000000,355.000000,4.259268e+00,10.000000,0.883100,17.347941,1.000000,7.001000,23.000000,9.000000,5.000000,5.000000,4.000000,3.000000,5.000000,4.000000,1.000000,1.101527e+02


In [44]:
le2 = preprocessing.LabelEncoder()
df3.mvar12 = le2.fit_transform(df3.mvar12)

In [45]:
df3["supplementary"] = df.mvar49
df3["elite"] = df.mvar50
df3["credit"] = df.mvar51

In [46]:
df3.elite = df3.elite*2
df3.credit = df3.credit*3

In [47]:
# 0:no card;  1:supplementary;  2:elite;  3:credit
df3["card_accepted"] = df3.supplementary + df3.credit + df3.elite

In [48]:
df3.drop(["supplementary", "elite", "credit"], axis=1, inplace=True)

In [49]:
df3.head()

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,...,mvar14,mvar15,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,spend,card_accepted
0,0,4,18,0.068737,0.7969,0.1831,12.728538,1,6.0899,12,...,1,0,0,0,0,0,0,0,0.321125,0
1,2,4,19,0.133874,0.0678,0.0000,14.886743,1,4.2016,17,...,1,0,0,0,0,0,0,0,-0.133348,0
2,2,4,35,-0.629323,0.7551,0.2986,13.119672,0,3.3760,10,...,1,1,0,0,0,0,0,0,-0.008518,0
3,0,4,274,0.381398,0.7969,0.5297,14.723799,1,4.1082,15,...,1,0,1,0,0,1,0,0,-0.072286,3
4,0,6,1,-0.984324,2.3238,0.0000,14.886743,0,0.0000,17,...,2,0,0,0,0,0,0,0,0.123923,1


In [50]:
x_df3 = df3.ix[:, :-1]
y_df3 = df3.ix[:, -1]

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [51]:
smote_enn = SMOTEENN(kind_smote='svm',ratio=0.4)

TypeError: __init__() got an unexpected keyword argument 'k'

In [52]:
print np.bincount(y_df3)

[31122  3447  2654  2777]


In [140]:
x_df3_resampled, y_df3_resampled = smote_enn.fit_sample(x_df3, y_df3)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/base.py:306: UserWarning: The target type should be binary.
  warnings.warn('The target type should be binary.')
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/base.py:306: UserWarning: The target type should be binary.
  warnings.warn('The target type should be binary.')


In [141]:
print np.bincount(y_df3_resampled)

[14821    23  9835  2777]


In [142]:
x_df3_resampled, y_df3_resampled = smote_enn.fit_sample(x_df3_resampled, y_df3_resampled)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/base.py:306: UserWarning: The target type should be binary.
  warnings.warn('The target type should be binary.')
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/base.py:306: UserWarning: The target type should be binary.
  warnings.warn('The target type should be binary.')


In [143]:
print np.bincount(y_df3_resampled)

[11693  3383  9581  2777]


In [144]:
x_df3_resampled, y_df3_resampled = smote_enn.fit_sample(x_df3_resampled, y_df3_resampled)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/base.py:306: UserWarning: The target type should be binary.
  warnings.warn('The target type should be binary.')
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/base.py:306: UserWarning: The target type should be binary.
  warnings.warn('The target type should be binary.')


In [145]:
print np.bincount(y_df3_resampled)

[10356  3383  9536  2228]


In [146]:
x_df4 = x_df3_resampled
y_df4 = y_df3_resampled

In [147]:
x_train, x_test, y_train, y_test = train_test_split(x_df4, y_df4, test_size=0.3, random_state=42)

In [118]:
df3_majority = df3[df3.card_accepted==0]
df3_supp = df3[df3.card_accepted==1]
df3_elite = df3[df3.card_accepted==2]
df3_credit = df3[df3.card_accepted==3]

In [109]:
df3_majority_downsampled = resample(df3_majority,
                                    replace=False,
                                    n_samples=18000,
                                    random_state=123)

In [110]:
df3_supp_upsampled = resample(df3_supp,
                              replace=True,
                              n_samples=9000,
                              random_state=123)

In [111]:
df3_elite_upsampled = resample(df3_elite,
                               replace=True,
                               n_samples=9000,
                               random_state=123)

In [112]:
df3_credit_upsampled = resample(df3_credit,
                                replace=True,    
                                n_samples=9000,     
                                random_state=123)

In [113]:
df4 = pd.concat([df3_majority_downsampled, df3_supp_upsampled, df3_elite_upsampled, df3_credit_upsampled])
df4.shape

(45000, 21)

In [114]:
train, test= train_test_split(df4, test_size=0.3)

In [115]:
x_train = train.ix[:, :-1]
y_train = train.ix[:, -1]
x_test = test.ix[:, :-1]
y_test = test.ix[:, -1]

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


In [196]:
rfc = RandomForestClassifier(n_estimators=100, class_weight={0:0.1, 1:0.3, 2:0.3, 3:0.3}, oob_score=True)

In [134]:
rfc.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True,
            class_weight={0: 0.1, 1: 0.3, 2: 0.3, 3: 0.3},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [135]:
y_predicted = rfc.predict(x_test)

In [136]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.884394447104
[[3772  221  290  170]
 [ 341 2245   19   18]
 [ 158    0 2564   17]
 [ 190   11   14 2504]]
             precision    recall  f1-score   support

          0       0.85      0.85      0.85      4453
          1       0.91      0.86      0.88      2623
          2       0.89      0.94      0.91      2739
          3       0.92      0.92      0.92      2719

avg / total       0.88      0.88      0.88     12534



In [198]:
param_grid = {
    'n_estimators': [100, 200],
    'max_features': ['auto', 'log2'],
    'min_samples_leaf': [1, 10, 50]
}

In [199]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid)

In [200]:
CV_rfc.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True,
            class_weight={0: 0.1, 1: 0.3, 2: 0.3, 3: 0.3},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200], 'max_features': ['auto', 'log2'], 'min_samples_leaf': [1, 10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [201]:
print CV_rfc.best_params_
print CV_rfc.best_score_

{'max_features': 'log2', 'n_estimators': 200, 'min_samples_leaf': 1}
0.93586152812


In [148]:
rfc_tuned = RandomForestClassifier(n_estimators=200,
                                   max_features='log2', 
                                   class_weight={0:0.1, 1:0.3, 2:0.3, 3:0.3})

In [149]:
rfc_tuned.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True,
            class_weight={0: 0.1, 1: 0.3, 2: 0.3, 3: 0.3},
            criterion='gini', max_depth=None, max_features='log2',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [150]:
y_predicted = rfc_tuned.predict(x_test)

In [151]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.9430139851
[[3002    3   99   23]
 [   1 1005    0    0]
 [ 136    0 2719    3]
 [ 142    0   29  489]]
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      3127
          1       1.00      1.00      1.00      1006
          2       0.96      0.95      0.95      2858
          3       0.95      0.74      0.83       660

avg / total       0.94      0.94      0.94      7651



In [152]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=3,
    min_child_weight=1,
    objective= 'multi:softmax',
 )

In [153]:
param_grid = {
    'n_estimators': [200, 300, 400],
    'learning_rate': [0.1, 0.01, 0.015]
}

In [43]:
CV_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid)

In [44]:
CV_xgb.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=10, min_child_weight=1, missing=None,
       n_estimators=100, nthread=-1, objective='multi:softmax', seed=0,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [200, 300, 400], 'learning_rate': [0.1, 0.01, 0.015]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [45]:
print CV_xgb.best_params_
print CV_xgb.best_score_

{'n_estimators': 400, 'learning_rate': 0.1}
0.796598639456


In [154]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=10, min_child_weight=1, missing=None,
       n_estimators=300, nthread=-1, objective='multi:softprob', seed=0,
       silent=True, subsample=1)

In [155]:
y_predicted = xgb.predict(x_test)

In [156]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.953208730885
[[3014    4   80   29]
 [   1 1005    0    0]
 [ 101    0 2750    7]
 [ 103    0   33  524]]
             precision    recall  f1-score   support

          0       0.94      0.96      0.95      3127
          1       1.00      1.00      1.00      1006
          2       0.96      0.96      0.96      2858
          3       0.94      0.79      0.86       660

avg / total       0.95      0.95      0.95      7651



In [ ]:
bbc = BalancedBagg

In [157]:
test_df= pd.read_csv("Leaderboard_Dataset.csv")
test_df.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,50001,Charge,2,0,4,18,460,0.2495,0.2310,180512,...,32886.89,11345.47,7914.21,7704.62,0,0,0,0,0,0
1,50002,Charge,2,0,0,45,381,1.9329,0.7196,11295211,...,104722.52,255544.59,116216.40,152673.00,1,0,0,0,0,0
2,50003,Charge,0,0,4,3,0,0.7969,0.0000,45161,...,19689.03,0.00,0.00,0.00,0,0,0,0,0,0
3,50004,Charge,1,18711,9,89,432,0.1016,0.5074,420972,...,13154.43,7671.62,3357.24,2651.22,0,1,0,0,0,0
4,50005,Charge,0,21384,4,7,0,0.7969,0.2310,880618,...,32071.78,15376.01,14390.97,0.00,0,0,0,0,0,0


In [158]:
test_income = test_df["mvar9"][test_df.mvar9>0]
test_income= test_income.sort_values(ascending=True).reset_index()
median_income= test_income.median()[1]
mean_income = test_income.mean()[1]
test_df.mvar9.replace(to_replace=0, value=mean_income, inplace=True)
test_spend = test_df.mvar36 + test_df.mvar37 + test_df.mvar38 + test_df.mvar39

In [159]:
test_df.drop(["cm_key", "mvar1", "mvar3"], axis=1, inplace=True)

In [160]:
test_df.head()

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,...,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,2,4,18,460,0.2495,0.2310,180512.0,0,5.0612,Personal services,...,32886.89,11345.47,7914.21,7704.62,0,0,0,0,0,0
1,2,0,45,381,1.9329,0.7196,11295211.0,0,4.0930,Misc services,...,104722.52,255544.59,116216.40,152673.00,1,0,0,0,0,0
2,0,4,3,0,0.7969,0.0000,45161.0,0,2.3941,Misc services,...,19689.03,0.00,0.00,0.00,0,0,0,0,0,0
3,1,9,89,432,0.1016,0.5074,420972.0,0,1.5442,Consumables,...,13154.43,7671.62,3357.24,2651.22,0,1,0,0,0,0
4,0,4,7,0,0.7969,0.2310,880618.0,0,2.6261,Non durable goods,...,32071.78,15376.01,14390.97,0.00,0,0,0,0,0,0


In [161]:
test_dfa= test_df.ix[:,0:13]

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [162]:
test_dfb= test_df.ix[:, 37:43]

In [163]:
test_df= pd.concat([test_dfa, test_dfb], axis=1)

In [164]:
le = preprocessing.LabelEncoder()
test_df.mvar12 = le.fit_transform(test_df.mvar12)

In [165]:
test_df.head(2)

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,2,4,18,460,0.2495,0.2310,180512.0,0,5.0612,13,12,1,0,0,0,0,0,0,0
1,2,0,45,381,1.9329,0.7196,11295211.0,0,4.0930,10,13,1,0,1,0,0,0,0,0


In [166]:
# test_df["mvar9"]= np.log(test_df["mvar9"])
std_scale = preprocessing.StandardScaler().fit(test_df.mvar9)
test_df.mvar9 = std_scale.transform(test_df.mvar9)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [167]:
test_df["spend"]=test_spend

In [168]:
std_scale = preprocessing.StandardScaler().fit(test_df.spend)
test_df.spend = std_scale.transform(test_df.spend)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [169]:
std_scale = preprocessing.StandardScaler().fit(test_df.mvar6)
test_df.mvar6 = std_scale.transform(test_df.mvar6)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [170]:
test_df.head()

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,spend
0,2,4,18,-0.487386,0.2495,0.2310,-0.491540,0,5.0612,13,12,1,0,0,0,0,0,0,0,-0.288141
1,2,0,45,-0.572347,1.9329,0.7196,2.396835,0,4.0930,10,13,1,0,1,0,0,0,0,0,0.788464
2,0,4,3,-0.982092,0.7969,0.0000,-0.526714,0,2.3941,10,10,1,0,0,0,0,0,0,0,-0.364091
3,1,9,89,-0.517499,0.1016,0.5074,-0.429052,0,1.5442,3,11,1,1,0,1,0,0,0,0,-0.350578
4,0,4,7,-0.982092,0.7969,0.2310,-0.309604,0,2.6261,11,14,6,0,0,0,0,0,0,0,-0.284382


In [183]:
test_result= rfc_tuned.predict(test_df)

In [184]:
test_result

array([0, 0, 0, ..., 0, 0, 0])

In [185]:
test_df2 = pd.read_csv("Leaderboard_Dataset.csv")

In [186]:
result= pd.DataFrame(
{
    "card_accepted": test_result,
    "cm_key": test_df2.cm_key
    
})

In [187]:
result.shape

(10000, 2)

In [188]:
result.card_accepted.replace(to_replace=0, value="No_card", inplace=True)

In [189]:
result.card_accepted.replace(to_replace=1, value="Supp", inplace=True)
result.card_accepted.replace(to_replace=2, value="Elite", inplace=True)
result.card_accepted.replace(to_replace=3, value="Credit", inplace=True)

In [190]:
submission= result[result.card_accepted != "No_card"]

In [180]:
submission = result[result.card_accepted != "Credit"]

In [191]:
submission.shape

(892, 2)

In [192]:
submission= submission.reindex_axis(sorted(submission.columns, reverse=True), axis=1)

In [193]:
submission= submission[:1000]

In [194]:
submission.head()

,cm_key,card_accepted
4,50005,Elite
15,50016,Elite
51,50052,Elite
53,50054,Elite
58,50059,Elite


In [195]:
submission.to_csv("Reckless_Serenade_IITRoorkee_38.csv", header=False, index=False)